# FPL Model Notebook

## Set Up

In [1]:
import pandas as pd
from pulp import *
pd.set_option("display.max_columns", 50)

#### Importing Future Fixture Difficulty and Previous Gameweek Information

In [2]:
df_fix = pd.read_csv('team_fix.csv')
df_fix.head(5)

,team,id,GW17,GW18,GW19,GW20,GW21,GWSUM
0,Arsenal,1,4,2,2,3,4,15
1,Aston Villa,2,2,2,4,2,2,12
2,Brentford,3,5,2,3,4,3,17
3,Brighton,4,3,4,2,5,3,17
4,Burnley,5,2,3,3,4,2,14


In [3]:
df_raw = pd.read_csv('merged_gw16.csv')
df_raw.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Eric Bailly,DEF,Man Utd,0.0,0,0,0,0,0.0,286,6,0,0,0.0,0.0,2021-08-14T11:30:00Z,0,10,0,0,0,0,1,0,9363,1,5,0.0,0,0,0,0,50,True,0,1
1,Keinan Davis,FWD,Aston Villa,0.4,0,0,0,0,0.0,49,8,0,0,0.0,0.0,2021-08-14T14:00:00Z,0,18,0,0,0,0,1,0,169789,2,3,0.0,0,0,0,0,45,False,0,1
2,Ayotomiwa Dele-Bashiru,MID,Watford,0.0,0,0,0,0,0.0,394,8,0,0,0.0,0.0,2021-08-14T14:00:00Z,0,2,0,0,0,0,1,0,4092,2,3,0.0,0,0,0,0,45,True,0,1
3,James Ward-Prowse,MID,Southampton,2.3,0,0,20,0,30.5,341,4,3,0,5.2,21.6,2021-08-14T14:00:00Z,90,8,0,0,0,0,1,0,299682,1,3,0.0,2,0,0,0,65,False,0,1
4,Bruno Miguel Borges Fernandes,MID,Man Utd,4.4,0,3,61,0,35.9,277,6,1,3,20.1,106.2,2021-08-14T11:30:00Z,90,10,0,0,0,0,1,0,3381004,1,5,59.0,20,0,0,0,120,True,0,1


In [4]:
print(df_raw.columns)

Index(['name', 'position', 'team', 'xP', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes',
       'opponent_team', 'own_goals', 'penalties_missed', 'penalties_saved',
       'red_cards', 'round', 'saves', 'selected', 'team_a_score',
       'team_h_score', 'threat', 'total_points', 'transfers_balance',
       'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards',
       'GW'],
      dtype='object')


#### Trimming Tables

In [5]:
df_trim = df_raw[['name', 'team', 'minutes', 'element','total_points','value','GW']]
df_trim.head()

,name,team,minutes,element,total_points,value,GW
0,Eric Bailly,Man Utd,0,286,0,50,1
1,Keinan Davis,Aston Villa,0,49,0,45,1
2,Ayotomiwa Dele-Bashiru,Watford,0,394,0,45,1
3,James Ward-Prowse,Southampton,90,341,2,65,1
4,Bruno Miguel Borges Fernandes,Man Utd,90,277,20,120,1


In [6]:
df_player_info = df_raw[['name','position', 'team','minutes','total_points', 'element','value','GW']]
df_player_info_trim = df_player_info[(df_player_info['GW'] == 16)]
df_player_info_trim.head()

,name,position,team,minutes,total_points,element,value,GW
9089,Eric Bailly,DEF,Man Utd,16,1,286,49,16
9090,Keinan Davis,FWD,Aston Villa,0,0,49,45,16
9091,Ayotomiwa Dele-Bashiru,MID,Watford,0,0,394,45,16
9092,James Ward-Prowse,MID,Southampton,90,2,341,63,16
9093,Bruno Miguel Borges Fernandes,MID,Man Utd,87,3,277,116,16


#### Creating Previous Gameweek Groupings

In [7]:
df_prev_1_gw = df_trim[( (df_trim['GW'] == 16))  ]
df_1_group = df_prev_1_gw.groupby(['name']).mean()

df_prev_2_gw = df_trim[((df_trim['GW'] >= 15) & (df_trim['GW'] <= 16))  ]
df_2_group = df_prev_2_gw.groupby(['name']).mean()

df_prev_5_gw = df_trim[((df_trim['GW'] >= 12) & (df_trim['GW'] <= 16))  ]
df_5_group = df_prev_5_gw.groupby(['name']).mean()

df_prev_10_gw = df_trim[((df_trim['GW'] >= 7) & (df_trim['GW'] <= 16))  ]
df_10_group = df_prev_10_gw.groupby(['name']).mean()

df_prev_16_gw = df_trim[((df_trim['GW'] >= 1) & (df_trim['GW'] <= 16))  ]
df_16_group = df_prev_16_gw.groupby(['name']).mean()

#### Merging Tables

In [8]:
df1 = df_16_group.join(df_10_group, on='name',lsuffix="_16", rsuffix="_10").join(df_5_group, on='name', rsuffix="_5").join(df_2_group, on='name', lsuffix ='_5', rsuffix="_2").join(df_1_group, on='name',lsuffix ='_2', rsuffix="_1")

df2 = df1.merge(df_player_info_trim, on='name',suffixes=('_1', ''))

df3 = df2.merge(df_fix, on ='team')

df = df3.drop(columns=['minutes_16', 'element_16', 'value_16','GW_16','minutes_10', 'element_10', 'value_10','GW_10','minutes_5', 'element_5', 'value_5','GW_5','minutes_2', 'element_2', 'value_2','GW_2','minutes_1', 'element_1', 'value_1','GW_1'])

df.head()

,name,total_points_16,total_points_10,total_points_5,total_points_2,total_points_1,position,team,minutes,total_points,element,value,GW,id,GW17,GW18,GW19,GW20,GW21,GWSUM
0,Aaron Cresswell,3.25,2.7,0.8,0.0,0.0,DEF,West Ham,0,0,411,54,16,19,4,2,2,2,3,13
1,Ajibola Alese,0.00,0.0,0.0,0.0,0.0,DEF,West Ham,0,0,644,40,16,19,4,2,2,2,3,13
2,Alex Kral,0.00,0.0,0.0,0.0,0.0,MID,West Ham,0,0,589,44,16,19,4,2,2,2,3,13
3,Alphonse Areola,0.00,0.0,0.0,0.0,0.0,GK,West Ham,0,0,489,47,16,19,4,2,2,2,3,13
4,Andriy Yarmolenko,0.50,0.2,0.2,0.0,0.0,MID,West Ham,0,0,412,51,16,19,4,2,2,2,3,13


In [9]:
print(df.columns)

Index(['name', 'total_points_16', 'total_points_10', 'total_points_5',
       'total_points_2', 'total_points_1', 'position', 'team', 'minutes',
       'total_points', 'element', 'value', 'GW', 'id', 'GW17', 'GW18', 'GW19',
       'GW20', 'GW21', 'GWSUM'],
      dtype='object')


## Calculating Ranking System 

In [10]:
df['calcpoints'] = (
    (df['total_points_16'] * 1) + 
    (df['total_points_10'] * 2) + 
    (df['total_points_5'] * 3) + 
    (df['total_points_2'] * 1) -
    (df['GWSUM'] * 1)-
    (df['GW17'] * 0.5)    
)
df.sort_values(by='calcpoints',ascending=False).head(20)

,name,total_points_16,total_points_10,total_points_5,total_points_2,total_points_1,position,team,minutes,total_points,element,value,GW,id,GW17,GW18,GW19,GW20,GW21,GWSUM,calcpoints
417,Mohamed Salah,10.000000,10.3,8.6,7.0,8.0,MID,Liverpool,90,8,233,131,16,11,2,3,2,3,5,15,47.400000
427,Trent Alexander-Arnold,6.750000,7.3,8.8,9.0,9.0,DEF,Liverpool,90,9,237,81,16,11,2,3,2,3,5,15,40.750000
491,Bernardo Mota Veiga de Carvalho e Silva,5.625000,6.7,8.2,10.5,6.0,MID,Man City,90,6,261,76,16,12,2,2,3,2,4,13,40.125000
303,Emmanuel Dennis,5.250000,6.1,8.8,5.5,9.0,FWD,Watford,90,9,450,57,16,18,2,2,3,4,3,14,34.350000
376,James Maddison,4.312500,5.8,8.8,8.5,16.0,MID,Leicester,87,16,212,67,16,9,3,3,5,4,2,17,32.312500
475,Mason Mount,4.812500,6.2,7.0,9.5,6.0,MID,Chelsea,90,6,138,75,16,6,3,3,3,3,4,16,30.212500
510,Raheem Sterling,3.312500,3.9,6.8,9.0,11.0,MID,Man City,90,11,255,106,16,12,2,2,3,2,4,13,26.512500
555,Conor Gallagher,5.437500,5.5,5.0,8.5,15.0,MID,Crystal Palace,90,15,144,61,16,7,2,2,3,2,4,13,25.937500
429,Virgil van Dijk,5.187500,4.9,6.8,6.0,5.0,DEF,Liverpool,90,5,229,66,16,11,2,3,2,3,5,15,25.387500
400,Andrew Robertson,3.625000,4.6,6.4,6.5,8.0,DEF,Liverpool,90,8,234,70,16,11,2,3,2,3,5,15,22.525000


## Linear Programming

In [11]:
# Helper variables
POS = df.position.unique()
CLUBS = df.team.unique()
BUDGET = 1003
pos_available = {
    'DEF': 5,
    'FWD': 3,
    'MID': 5,
    'GK': 2,
}

# Initialize Variables
names = [df.name[i] for i in df.index]
teams = [df.team[i] for i in df.index]
positions = [df.position[i] for i in df.index]
prices = [df.value[i] for i in df.index]
target = [df.calcpoints[i] for i in df.index]
players = [LpVariable("player_" + str(i), cat="Binary") for i in df.index]

In [12]:
# Initialize the problem
prob = LpProblem("FPL Player Choices", LpMaximize)

C:\Users\zacjo\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [13]:
# Define the objective
prob += lpSum(players[i] * target[i] for i in range(len(df))) # Objective

In [14]:
# Build the constraints
prob += lpSum(players[i] * df.value[df.index[i]] for i in range(len(df))) <= BUDGET # Budget Limit

for pos in POS:
  prob += lpSum(players[i] for i in range(len(df)) if positions[i] == pos) == pos_available[pos] # Position Limit

for club in CLUBS:
  prob += lpSum(players[i] for i in range(len(df)) if teams[i] == club) <= 3 # Club Limit

In [15]:
# Solve the problem
prob.solve()

1

In [16]:
for v in prob.variables():
  if v.varValue != 0:
    name = df.name[int(v.name.split("_")[1])]
    club = df.team[int(v.name.split("_")[1])]
    position = df.position[int(v.name.split("_")[1])]
    #point = df.total_points[int(v.name.split("_")[1])]
    target = df.calcpoints[int(v.name.split("_")[1])]
    price = df.value[int(v.name.split("_")[1])]
    print(name, position, club, target, price, sep=" | ")

Ezri Konsa Ngoyo | DEF | Aston Villa | 20.900000000000006 | 49
Ollie Watkins | FWD | Aston Villa | 13.162500000000001 | 75
David de Gea | GK | Man Utd | 18.825000000000003 | 51
Armando Broja | FWD | Southampton | 6.287499999999998 | 50
Emmanuel Dennis | FWD | Watford | 34.35 | 57
James Maddison | MID | Leicester | 32.3125 | 67
Mohamed Salah | MID | Liverpool | 47.4 | 131
Trent Alexander-Arnold | DEF | Liverpool | 40.75 | 81
Virgil van Dijk | DEF | Liverpool | 25.387500000000003 | 66
Antonio Rüdiger | DEF | Chelsea | 18.762499999999996 | 61
Mason Mount | MID | Chelsea | 30.2125 | 75
Bernardo Mota Veiga de Carvalho e Silva | MID | Man City | 40.125 | 76
Conor Gallagher | MID | Crystal Palace | 25.9375 | 61
Aaron Ramsdale | GK | Arsenal | 18.96666666666667 | 50
Kieran Tierney | DEF | Arsenal | 18.599999999999994 | 47
